In [1]:
! pip install  xmltodict
! pip install --upgrade albumentations

     |████████████████████████████████| 81kB 3.9MB/s 
     |████████████████████████████████| 37.6MB 1.8MB/s 
     |████████████████████████████████| 952kB 28.2MB/s 
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [2]:
import os 
import ast
import pandas as pd
import numpy as np 
import shutil
from tqdm import tqdm # for progress bar
from pprint import pprint 
import json
import xmltodict
import ast
import torch
import torch.utils.data
from PIL import Image
import cv2
import albumentations as A

In [3]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt  # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.7.0+cu101 CPU


In [4]:
transform = A.Compose([
    A.Resize(512, 512)
], bbox_params=A.BboxParams(format='yolo'))

In [6]:
with open('/content/drive/MyDrive/torch/archive/test_zip/test/apple_79.xml') as fd:
    bb_file = json.loads(json.dumps(xmltodict.parse(fd.read())))
    pprint(bb_file)

{'annotation': {'filename': 'apple_79.jpg',
                'folder': 'test',
                'object': {'bndbox': {'xmax': '641',
                                      'xmin': '77',
                                      'ymax': '716',
                                      'ymin': '183'},
                           'difficult': '0',
                           'name': 'apple',
                           'pose': 'Unspecified',
                           'truncated': '0'},
                'path': 'C:\\tensorflow1\\models\\research\\object_detection\\images\\test\\apple_79.jpg',
                'segmented': '0',
                'size': {'depth': '3', 'height': '0', 'width': '0'},
                'source': {'database': 'Unknown'}}}


In [7]:
# новый вариант с ресайзом

classes = {'apple' : 0, 'orange' : 1, 'banana' : 2, 'mixed' : 3}
folder = {'train' : 'train', 'test' : 'validation'}
OUTPUT_PATH = '/content/drive/MyDrive/torch/fruit_cropped1'

def prepare_data(path, counter=0):
    images = []
    for filename in tqdm(os.listdir(path), total=len(os.listdir(path))): # last file .ipynd, not found
        sp_filename = filename.split('.')

        if sp_filename[1]=='xml':
            fruit_data = []
            image = cv2.imread(f"{path}/{sp_filename[0]}.jpg")
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 

            with open(f'{path}/{filename}') as fd:   # open bbox
                bb_file = json.loads(json.dumps(xmltodict.parse(fd.read())))
                image_path = f"{path}/{bb_file['annotation']['filename']}"
                data_type = folder[bb_file['annotation']['folder']] # train or validation
                size_h = image.shape[0] # cv2 format of shape
                size_w = image.shape[1]
                if type(bb_file['annotation']['object']) == list:
                    for bboxes in bb_file['annotation']['object']:
                        xmin = float(bboxes['bndbox']['xmin'])
                        xmax = float(bboxes['bndbox']['xmax'])
                        ymin = float(bboxes['bndbox']['ymin'])
                        ymax = float(bboxes['bndbox']['ymax']) # after this step we need to format bb to our format image
                        # ptype = classes[bboxes['name']]
                        w = xmax - xmin
                        h = ymax - ymin
                        x_center = xmin + w / 2 - 1 # -1 use when convert from vok to yolo
                        y_center = ymin + h / 2 - 1
                        w /= size_w  # norm
                        h /= size_h
                        x_center /= size_w 
                        y_center /= size_h

                        fruit_data.append([x_center, y_center, w, h, bboxes['name']])
                        
                else:
                    bboxes = bb_file['annotation']['object']
                    xmin = float(bboxes['bndbox']['xmin'])
                    xmax = float(bboxes['bndbox']['xmax'])
                    ymin = float(bboxes['bndbox']['ymin'])
                    ymax = float(bboxes['bndbox']['ymax']) # after this step we need to format bb to our format image
                    # ptype = classes[bboxes['name']]
                    w = xmax - xmin
                    h = ymax - ymin
                    x_center = xmin + w / 2 - 1  # -1 use when convert from vok to yolo
                    y_center = ymin + h / 2 - 1 
                    w /= size_w  # norm
                    h /= size_h
                    x_center /= size_w 
                    y_center /= size_h

                    fruit_data.append([x_center, y_center, w, h, bboxes['name']])
            transformed = transform(image=image, bboxes=fruit_data)
            transformed_image = transformed['image']
            transformed_bboxes = transformed['bboxes'] # to format, wich our network will understand
            
            for i, bbox in enumerate(transformed_bboxes):
                transformed_bboxes[i] = [classes[bbox[-1]]] + list(bbox[:-1])

            # now bbox in format [class, x_center, y_center, w, h]
            fruit_data = np.array(transformed_bboxes)
            np.savetxt(  
                os.path.join(OUTPUT_PATH, f"labels/{data_type}/{sp_filename[0]}.txt"),
                # os.path.join(OUTPUT_PATH, f"labels/label_{counter}.txt"),
                fruit_data,
                fmt=["%d", "%f", "%f", "%f", "%f"] # no idea
                )
            cv2.imwrite(os.path.join(OUTPUT_PATH, f"images/{data_type}/{sp_filename[0]}.jpg"),
                        transformed_image) # save image
            counter += 1
    return counter # return counter to start counting from end
            


last_num = prepare_data('/content/drive/MyDrive/torch/archive/test_zip/test')
prepare_data('/content/drive/MyDrive/torch/archive/train_zip/train', last_num)

100%|██████████| 480/480 [06:38<00:00,  1.21it/s]


300

In [ ]:
!python /content/yolov5/train.py --img 512 --batch 10 --epochs 30 --data /content/drive/MyDrive/torch/fruits.yaml --cfg /content/yolov5/models/yolov5x.yaml --weights yolov5x.pt --name yolov5x1cr

In [ ]:
!python /content/yolov5/detect.py --source /content/drive/MyDrive/torch/fruit_cropped1/images/validation/apple_77.jpg --weights /content/yolov5/runs/train/yolov5x1cr/weights/best.pt